In [1]:
# Definition for singly-linked list.
# class ListNode:
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next = next

<b> 2. Add Two Numbers </b>  
Linked list, having an extra var to save the add up value if x + y > 10.   
Able to merge  while (l1 and l2),  while (l1), and while (l2) together into while (l1 or l2)  
Remember to check if the addup value >0, to add extra node.  
also keep the start node and a duplicate of start node. the duplicate are moving forward, but start node is a pointer that we can use to return the result at the very end

time complexity is O(max(m,n)). Assume that m and n represents the length of l1 and l2 respectively, the algorithm below iterates at most max(m, n) times.

In [2]:
def addTwoNumbers(l1, l2):
    addup = 0
    start = ListNode()
    node = start    # point to start,  NOT start.next which will break the connection
    while l1 or l2:
        x = l1.val if l1 else 0
        y = l2.val if l2 else 0
        val = x+y+addup
        if val >= 10:
            addup = 1
            val = val%10
        else:
            addup = 0
        node.next = ListNode(val)
        node = node.next  # move to next None
        l1=l1.next if l1 else None
        l2=l2.next if l2 else None
    if addup>0:
        node.next = ListNode(addup)
    return start.next

<b>21. Merge two Sorted Lists </b>


a <b>wrong</b> implementation here

In [4]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next
def mergeTwoLists_not_correct(l1, l2):
    start = ListNode()
    # this step causes the issue, it does not connect node and start, so later on the update to node not affect start at all
    node = start.next   
    while l1 and l2:
        if l1.val<=l2.val:
            node = l1
            l1 = l1.next
        else:
            node = l2
            l2 = l2.next 
        node = node.next
    if l1:
        node = l1
    if l2:
        node = l2
    return start.next

In [7]:
l1 = ListNode(1)
l1.next = ListNode(2)
l1.next.next = ListNode(3)
l2 = ListNode(1)
l2.next = ListNode(3)
l2.next.next = ListNode(4)
print(mergeTwoLists_not_correct(l1, l2))  # it returns nothing, because start.next is None, node does not affect start

None


In [8]:
def mergeTwoLists(l1, l2):
    start = ListNode()
    node = start   # important, node points to start in initial step, node is the worker
    while l1 and l2:
        if l1.val<=l2.val:
            # always update the next 
            node.next = l1
            l1 = l1.next
        else:
            node.next = l2
            l2 = l2.next 
        # now node points to the current one
        node = node.next
    if l1:
        node.next = l1
    if l2:
        node.next = l2
    return start.next

In [11]:
l1 = ListNode(1)
l1.next = ListNode(2)
l1.next.next = ListNode(3)
l2 = ListNode(1)
l2.next = ListNode(3)
l2.next.next = ListNode(4)
s = mergeTwoLists(l1, l2)
while s:
    print(s.val, end=",")
    s=s.next

1,1,2,3,3,4,

<b>138.Copy List with Random Pointer </b>

get confused when first looking at the question. I thought it is returnning a list, but it is returnning a DEEP COPU of the linked list.

way 1: using hashmap, first loop: create new node, duplicate the value  
second loop: duplicate the next and random relationship  

time complexity O(N)

In [12]:
def copyRandomList(head):
    """using hash map"""
    hash_map = {}
    m = n = head
    while m:
        # duplicate each node's value
        hash_map[m] = Node(m.val)
        m = m.next
    while n:
        # duplicate the next and random relationship
        hash_map[n].next = hash_map.get(n.next)
        hash_map[n].random = hash_map.get(n.random)
        n = n.next
    return hash_map.get(head)

this one is same as above, but reduce to one loop due to the usage of defaultdict. 
one attribute of defaultdict: if a key does not exist yet in the dict, when you call dict[key], that key wil lbe auto inserted into the dict

In [13]:

def copyRandomList_2(head):
        """using hash map with default dict"""
        hash_map = defaultdict(lambda: Node(0))
        hash_map[None] = None
        m = head
        while m:
            # duplicate each node's value
            hash_map[m].val = m.val
            # for a defaultdict, if the key not exit, when using dict[key], the key is auto inserted into the dictioanry.
            hash_map[m].next = hash_map[m.next]
            hash_map[m].random = hash_map[m.random]
            m = m.next
        
        return hash_map.get(head)

way 3: space O(1), time O(N), concern is it modified the passed parameter. 
basic idea is to copy the node and put the copied node next to the original node. 
then loop again to copy the random relationship.  
finally loop again the break and correct the next relationship  

here is a good explanation in the first comments of this [post](https://leetcode.com/problems/copy-list-with-random-pointer/discuss/43491/A-solution-with-constant-space-complexity-O(1)-and-linear-time-complexity-O(N))

In [14]:
def copyRandomList(head):
    """space O(1), time O(N), but it will modify the original linked list which could be a concern"""
    if not head:
        return None

    # duplicate the node value and attach to the neighbor
    dummy = head
    while dummy:
        next_node = dummy.next
        copy = Node(dummy.val)
        copy.next = dummy.next
        dummy.next = copy
        dummy = next_node

    # duplciate the random relationship
    dummy = head
    while dummy:
        dummy.next.random = dummy.random.next if dummy.random else None
        dummy = dummy.next.next 
    # reorder the next relationship
    dummy = head
    copy_head = head.next  # None head has been checked at first line
    while dummy:
        next_node = dummy.next.next
        copy_node = dummy.next
        dummy.next = next_node
        copy_node.next = next_node.next if next_node else None
        dummy = next_node

    return copy_head


<b>206.Reverse Linked list </b>

there are two ways to address it. one is iteraltive way, another is recursive

way 1: iterative  
example:  
1->2->3->4->5->null: new_start is 5, new_end is null, node is 1  
node is inserted between new_start and new_end.  
now new_start is still 5, new end should update to be 1, node is 2  
2->3->4->5->1->null  
node 2 is inserted into new start 5 and new end 1, and so on  
until node goes to new start 5  

In [16]:
def reverseList(head):
    '''iterative way'''
    new_head = None
    """
    example:
    1->2->3->4->5->null
    1->null, then 1 becomes the new head, head move to next, which is 2
    2->1->null, head connect to new head, 2 becomes the new head, head move to next, which is 3
    """
    while head:
        next_node = head.next
        head.next = new_head
        new_head = head
        head = next_node
    return new_head

In [20]:
# class ListNode:
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next = next
def reverseList(head):   # -> this one can be generalized into question move m to n in a linked list
    """
    example: 1->2->3->4->5->null   => 0->1->2->3->4->5->null
    0->2->1->3->4->5->null
    0->3->2->1->4->5->null
    0->4->3->2->1->5->null
    0->5->4->3->2->1->null
    return 0->. which is 5
    """
    if not head or not head.next:
        return head
    dummy = ListNode(0)
    dummy.next=head
    # need to know start node: head, pre_node: dummy, and a moving_node
    pre_node = dummy
    moving_node = head.next
    while moving_node:
        next_node = moving_node.next
        head.next = moving_node.next
        moving_node.next = pre_node.next
        pre_node.next = moving_node
        moving_node = next_node
    return dummy.next

way 2: recursive, same idea as above first way

In [18]:
def reverseList(head):  # => this one actually not too recursive, but easy to understand
    '''recursive way'''
    def revesere(head, new_head):
        if not head:
            return new_head
        """
        example:
        1->2->3->4->5->null
        initilize head to 1, new_head to None, save next_node 2, connect head 1 to new_head None 
        recurisive starts by assigning next_node to head, and head to new_head
        
        """
        next_node = head.next
        head.next = new_head
        # same idea, new_head = head, head = next_node
        return revesere(next_node, head)

    # initialize new_head to none
    return revesere(head, None)


<b> 92.reverse Linked List II </b>

there are two ways of iterative.  
<b>way 1</b>: need to know the pre_node, start node and end node.  
example: 1->2->3->4->5->none  reverse from position 2 to 4, final should be 1->4->3->2->5->None   

5 is called new_head (next of end node), 2 is called head (start node), 1 is called pre_node  
step 1: 3->4->5->none  
        1->2->5->none  next_node is 3, head 2 link to the new_head 5, 2 becomes new_head, head move to next_node  
step 2: 4->5->None  
        3->2->5->none next_node is 4, head 3 link to the new_head 2, 3 becomes new_head, head move to next_node  
        1->2->5->none  
step 3: 5->None  
        4->3->2->5->none  
        1->2->5->none  
last step: pre_node 1 link to new head 4  
downside is you need to know pre_node, start_node, and end_node => cant leverage the m-n to get from nth node to mth node, and need to keep an eye on last step pre_node 

<b>way 2</b> need to pre_node, start node, easier to generalize this way to other scenarios. 
example: 1->2->3->4->5->none  reverse from position 2 to 4, final should be 1->4->3->2->5->None   

create a dummy node 0, 0->1->2->3->4->5->none dummy.next is to check where the head should be  
get pre_node 1, start_node 2, and a movin_node 3    
step 1: 2->4->5->None   =>.  0->1->3->2->4->5->none  moving node 3 is put after pre_node 1, start_node 2 point to moving node's next 4, moving node update to 4  
step 2: 2->5->none. =>. 0->1->4->3->2->5->none. moving node 4 is put after pre_node 1, start_node 2 point to moving node's next 5, moving node update to 5  

Here the loop is reaching the m-n times, so reverse finished.    
and dummy.next is the head should return, no need the last step as above way 1  

In [19]:
def reverseBetween(head, m, n) -> ListNode:
    """better than the other way of iterative: only need to get mth node, and using m-n to reverse, no need to know nth node in advance"""
    if not head:
        return
    dummy = ListNode(0)
    # a dummy node is needed, because cant return head directly, head can be moved to later position
    # return dummy.next will always return the head because the pre_node is updating 
    dummy.next = head
    pre_node = dummy
    for _ in range(m-1):  
        # loop m-1 times to get to the node before the mth node
        # because pre_node start at dummy which is before head, so need to loop m-1 times, if pre_node start at head, loop m-2 times
        # however, cant guarantee m-2>=0, so good to have dummy
        pre_node = pre_node.next
    # start is the mth node
    start = pre_node.next    
    moving_node = start.next
    for _ in range(n-m):
        # pre_node and start node never change
        # moving mode is moving to next one
        next_node = moving_node.next
        start.next = moving_node.next
        moving_node.next = pre_node.next
        pre_node.next = moving_node
        moving_node = next_node
    return dummy.next

another iterative way can solve the problem too but it is not as clean as this one.  
see the leetcode submit [history](https://leetcode.com/submissions/detail/375193188/)

<b>25..Reverse Nodes in K-group </b>

<b>way1</b>: iterative to have time O(n), space O(1).  
here using the <b>way 2 iterative</b> mentioned above: pre_node, start_node hold steady, moving node moving     

In [24]:
def reverseKGroup(head, k):
    if not head or not head.next or k ==1:
        return head
    dummy=ListNode()
    dummy.next = head

    count = 1
    pre_node = dummy
    start_node=head
    moving_node=head.next 

    # not just head, like in 1->2->3->none,k=2, when at 3, count is 3, then continue head = head.next, now count is 4, but head is null, should not get into the for loop. check head.next can avoid it
    while head.next:  
        count+=1
        head = head.next
        if count%k==0 and start_node:
            moving_node = start_node.next
            for _ in range(k-1): # k-1 time is enough
                next_node = moving_node.next
                start_node.next = moving_node.next
                moving_node.next = pre_node.next
                pre_node.next = moving_node
                moving_node = next_node
            # update to the next reverse chunk
            pre_node = start_node
            start_node = moving_node
            head = pre_node  # the original head before the if check has now changed, so need to update here too
    return dummy.next       

In [25]:
# 1->2->3.  => 2->1->3
l1 = ListNode(1)
l1.next = ListNode(2)
l1.next.next = ListNode(3)
a = reverseKGroup(l1, 2)
while a:
    print(a.val, end=", ")
    a = a.next

2, 1, 3, 

the <b>way 1 iterative</b> way. The connection step is smart, to jump to the right start of next chunk.  
need to know start node, end node of the range. end node is updating, moving node is updating too

In [ ]:
def reverseKGroup(head, k) -> ListNode:
    """way 1 iterative"""
    dummy = ListNode()
    dummy.next = head
    connect_node = dummy
    left = right = head

    while True:
        count = 0
        while right and count<k:
            count += 1
            right = right.next
        if count == k:
            # right is point at the knth+1 position
            tail = right
            curr = left
            for _ in range(k):  # change k times
                next_node = curr.next
                curr.next = tail
                tail = curr
                curr = next_node
            # here the start of the nodes chunk should be linked
            # the last node reversed above is the head of this chunk
            # this connecting step is smart
            connect_node.next = tail
            connect_node = left # the original left node now is the end of this chunk, serves perfectly as the pre node of next chunk
            left = curr # start of next chunk, or right, which is the kn+1 node, same
        else:
            return dummy.next
